## Carteira com Cotização

#### Functions:

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import wget
from datetime import datetime
import requests
#import cryptocompare as cryp


import os
#os.chdir('C:\\Users\\fabio\\OneDrive\\Área de Trabalho\\asset alocation')

In [7]:
#asset_sheet = pd.read_excel('acao.xlsx')


def crip_quotation(symbol, ini_date, comparison_symbol='BRL', all_data=True, limit=1, aggregate=1, exchange=''):
    from datetime import datetime
    url = 'https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&limit={}&aggregate={}'\
            .format(symbol.upper(), comparison_symbol.upper(), limit, aggregate)
    if exchange:
        url += '&e={}'.format(exchange)
    if all_data:
        url += '&allData=true'
    page = requests.get(url)
    data = page.json()['Data']
    df = pd.DataFrame(data)
    df['timestamp'] = [datetime.fromtimestamp(d) for d in df.time]
    
    df['timestamp'] = df['timestamp'].dt.normalize()
    df = df.set_index('timestamp').drop(['time','high','low','open','volumefrom','volumeto','conversionType','conversionSymbol'],axis=1)
    df = df[ini_date:]
    
    return df

#################################################################################################################33

def portfolio_prices(asset_sheet):
    
    df = pd.DataFrame()
    asset_sheet.set_index('ativo', inplace=True)
    
    for i in asset_sheet.index:
        if i == 'caixa':
            df[i] = 1
            
        elif asset_sheet.index[0]=='BTC':
            df[i] = crip_quotation(i, asset_sheet['data ini'][0])['close']
            
        else:
            df[i] = yf.download(i, asset_sheet['data ini'][0], asset_sheet['data fim'][0])['Adj Close']
        
    return(df)

#################################################################################################################33

def portfolio_prices_dynamic(xlsx_name):
    
    df = pd.DataFrame()
    i=1
    
    while True:
        try:            
            x = pd.read_excel(xlsx_name+'.xlsx', f'Planilha{i}')
            x =  portfolio_prices(x)
            df = df.append(x)
            i += 1

        except:
            break
    
    df = df[~df.index.duplicated(keep='first')]
            
    return(df)

#################################################################################################################33

def portfolio_index_open(asset_sheet):
    
    asset_weights = asset_sheet.set_index('ativo')['peso']
    df = portfolio_prices(asset_sheet)
    df = df/df.iloc[0,:]*asset_weights*100
    
    return(df)


###################################################################################################

def portfolio_index_open_dynamic(xlsx_name):

    df = pd.DataFrame()
    i=1

    while True:
        try:
            x = pd.read_excel(xlsx_name+'.xlsx', f'Planilha{i}')
            x = portfolio_index_open(x)
            df = df.append(x)
            i += 1
        except:
            break
    df = df[~df.index.duplicated(keep='first')]
    return(df)


###############################################################################################################333

def portfolio_index (xlsx_name):
    
    df = portfolio_index_open_dynamic(xlsx_name)
    
    return(pd.DataFrame(df.sum(axis=1)))



###############################################################################################################

def rf_prices(portfolio_items = 3):

    from datetime import datetime
    df = pd.DataFrame()
    for i in np.arange(0,portfolio_items):
        rf = pd.read_excel('rf.xlsx')
    
        link = rf['download'][i]
        arquivo = rf['arquivo'][i]
        aba = rf['ativo'][i]
        ini_date = rf['data ini'][i]

        wget.download(link)
        td = pd.read_excel(arquivo, aba,skiprows=[0]).set_index('Dia')
        os.remove(arquivo)

        td = td[ini_date:]
        
        td = td.set_index(pd.DataFrame([datetime.strptime(date, '%d/%m/%Y').date() for date in td.index[:]])[0])
        td = td['PU Compra Manhã']
        
        df[aba] = td

    return(df)

#####################################################################################################################

def rf_index_open(j):
    
    asset_sheet = pd.read_excel('rf.xlsx')
    asset_weights = asset_sheet.set_index('ativo')['peso']
    df = rf_prices(3)
    df = df/df.iloc[0,:]*asset_weights*100

    return(pd.DataFrame(df))

#####################################################################################################################

def rf_index(j):
    
    df = rf_index_open(j)
    
    return(pd.DataFrame(df.sum(axis=1)))



In [9]:
### baixar todos os ativos
n=1

df = pd.read_excel('pesos.xlsx').set_index('ativo')
lista = df.index

for i in np.arange(0,len(lista)):
    if i == 0:
        x = rf_index(0).rename({0:lista[i]},axis=1) 
    else:
        y = portfolio_index(lista[i]).rename({0:lista[i]},axis=1)  
        x = pd.concat([x,y],axis=1)
x = x.dropna()


###### criar o index
df = pd.DataFrame()
i=1

while True:
    try:            
        sheet_peso = pd.read_excel('pesos'+'.xlsx', f'Planilha{i}')
        pesos = sheet_peso['peso']
        ini = sheet_peso['data ini'][0]
        fim = sheet_peso['data fim'][0]
        aux = x[lista[0]]*pesos[0]+x[lista[1]]*pesos[1]+x[lista[2]]*pesos[2]+x[lista[3]]*pesos[3]+x[lista[4]]*pesos[4]+x[lista[5]]*pesos[5]
        aux = aux[ini:fim]
        aux = pd.DataFrame(aux)
        df = df.append(aux)
        i += 1

    except:
        break
    
    df = df[~df.index.duplicated(keep='first')]
    
df


##output
x['index'] = df[0]
x

[*********************100%***********************]  1 of 1 completed................] 90112 / 90112


,rf,acao,acao_us,commodities,valor,cripto,index


In [ ]:
x.to_excel('CONSOLIDADO_INDEX.xlsx')

In [ ]:
n=1

lista = ['rf', 'acao','acao_us','commodities','valor','cripto']

with pd.ExcelWriter('CONSOLIDADO_ABERTO.xlsx') as writer:
    for i in np.arange(0,len(lista)):
        if i == 0:
            x = pd.read_excel(lista[i]+'.xlsx', f'Planilha{n}')
            x = rf_prices(3)
            x.to_excel(writer, sheet_name=lista[i])
        else:
            y = pd.read_excel(lista[i]+'.xlsx', f'Planilha{n}')
            y = portfolio_prices(y)  
            y.to_excel(writer, sheet_name=lista[i])



In [ ]:
df = pd.DataFrame({})
ini_date = pd.to_datetime('2022-05-02')
end_date = pd.to_datetime('2023-06-01')

lista = ['^BVSP','^GSPC','BTC-USD'] #,'^DJCI']
for i in lista:
    df[i] = yf.download(i,ini_date, end_date)['Adj Close']
df = df/df.iloc[0,:]*100
df

df.to_excel('CONSOLIDADO_BENCHS.xlsx')


In [ ]:
#with pd.ExcelWriter('ACOES_ABERTO.xlsx') as writer:
#    for i in np.arange(1,4):
#        y = pd.read_excel('acao'+'.xlsx', f'Planilha{i}')
#        y = portfolio_prices(y)  
#        y.to_excel(writer, sheet_name=f'Planilha{i}')

In [ ]:
acao_aberto = portfolio_prices_dynamic('acao')

acao_aberto.to_excel('ACOES_ABERTO.xlsx')